In [1]:
# @title
!pip install datasets
!pip install evaluate
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 10.4 MB/s eta 0:00:00


In [2]:
from transformers import BertTokenizer, TFBertModel
from google.colab import drive
from datasets import load_from_disk

from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate

In [3]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# THIS PART IS FOR AFTER TRAINING ON THE INTERMEDIATE TASK.
sarcasm_dataset = load_from_disk('/content/gdrive/MyDrive/arrow-sarcasm-small-subset')
print(sarcasm_dataset)

DatasetDict({
    train: Dataset({
        features: ['label', 'comment'],
        num_rows: 30000
    })
    validate: Dataset({
        features: ['label', 'comment'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['label', 'comment'],
        num_rows: 10000
    })
})


In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
  return tokenizer(examples['comment'], padding = 'max_length', truncation = True)

tokenized_datasets = sarcasm_dataset.map(tokenize_function, batched = True)
print(tokenized_datasets)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'comment', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 30000
    })
    validate: Dataset({
        features: ['label', 'comment', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['label', 'comment', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10000
    })
})


In [6]:
# model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 2)
model = AutoModelForSequenceClassification.from_pretrained('/content/gdrive/MyDrive/orig_size', num_labels = 2)
training_args = TrainingArguments(output_dir = 'test_trainer', evaluation_strategy = 'epoch')

metric = evaluate.load('accuracy')

def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis = -1)
  return metric.compute(predictions = predictions, references = labels)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['test'],
    compute_metrics = compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.583600,0.573082,0.709800
2,0.483000,0.604459,0.721400
3,0.359700,0.933256,0.717700


TrainOutput(global_step=11250, training_loss=0.48474378458658857, metrics={'train_runtime': 9105.692, 'train_samples_per_second': 9.884, 'train_steps_per_second': 1.235, 'total_flos': 2.36799949824e+16, 'train_loss': 0.48474378458658857, 'epoch': 3.0})

In [7]:
trainer.evaluate()

{'eval_loss': 0.9332558512687683,
 'eval_accuracy': 0.7177,
 'eval_runtime': 312.4574,
 'eval_samples_per_second': 32.004,
 'eval_steps_per_second': 4.001,
 'epoch': 3.0}

In [8]:
trainer.save_model('/content/gdrive/MyDrive/sarcasm_detection_model')